<a href="https://colab.research.google.com/github/hrk022/Smart-Movie-Recommender/blob/main/Smart-Movie-Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [273]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [274]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [275]:
movies = movies.merge(credits, on="title")

In [276]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [277]:
movies = movies[["movie_id","title","cast","crew","keywords","genres","overview"]]

In [278]:
movies.isnull().sum()

,0
movie_id,0
title,0
cast,0
crew,0
keywords,0
genres,0
overview,3


In [279]:
movies.dropna(subset=['overview'],inplace=True)

<ipython-input-279-1606cb152e86>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.dropna(subset=['overview'],inplace=True)


In [280]:
movies.duplicated().sum()

np.int64(0)

In [281]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [282]:
import ast
def convert(obj):
  data = []
  for i in ast.literal_eval(obj):
    data.append(i["name"])
  return data

In [283]:
movies["genres"] = movies["genres"].apply(convert)
movies["keywords"] = movies["keywords"].apply(convert)

In [284]:
def convertor(obj):
  data = []
  flag = 0
  for i in ast.literal_eval(obj):
    if flag != 3:
      flag += 1
      data.append(i["name"])
    else:
      break
  return data

In [285]:
movies['cast'] = movies['cast'].apply(convertor)

In [286]:
movies["crew"][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [287]:
def director(obj):
  data = []
  for i in ast.literal_eval(obj):
    if i["job"] == "Director":
      data.append(i["name"])
  return data

In [288]:
movies['crew'] = movies["crew"].apply(director)

In [289]:
movies.head()

,movie_id,title,cast,crew,keywords,genres,overview
0,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],"[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],"[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],"[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]",Following the death of District Attorney Harve...
4,49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],"[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]","John Carter is a war-weary, former military ca..."


In [290]:
movies.iloc[0].overview

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [291]:
movies['overview'] = movies['overview'].astype(str).apply(lambda x:x.split())

In [292]:
movies['overview'] = movies['overview'].apply(lambda x:[i.replace(" ","") for i in x])

In [293]:
movies["genres"] = movies["genres"].apply(lambda x:[i.replace(" ","") for i in x])

In [294]:
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [295]:
movies["tags"] = movies["overview"] + movies["genres"] + movies["cast"] + movies["crew"]

In [296]:
movies.head()

,movie_id,title,cast,crew,keywords,genres,overview,tags
0,19995,Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[cultureclash, future, spacewar, spacecolony, ...","[Action, Adventure, Fantasy, ScienceFiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[ocean, drugabuse, exoticisland, eastindiatrad...","[Adventure, Fantasy, Action]","[Captain, Barbossa,, long, believed, to, be, d...","[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[spy, basedonnovel, secretagent, sequel, mi6, ...","[Action, Adventure, Crime]","[A, cryptic, message, from, Bond’s, past, send...","[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[dccomics, crimefighter, terrorist, secretiden...","[Action, Crime, Drama, Thriller]","[Following, the, death, of, District, Attorney...","[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[basedonnovel, mars, medallion, spacetravel, p...","[Action, Adventure, ScienceFiction]","[John, Carter, is, a, war-weary,, former, mili...","[John, Carter, is, a, war-weary,, former, mili..."


In [297]:
movies = movies.drop(columns= ["overview", "keywords","genres","crew", "cast"])

In [298]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words="english")

In [299]:
movies["tags"] = movies["tags"].apply(lambda x:" ".join(x))
vector = cv.fit_transform(movies["tags"]).toarray()

In [300]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [301]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [302]:
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
  words = nltk.word_tokenize(text)
  return " ".join([lemmatizer.lemmatize(word) for word in words])

In [303]:
movies["tags"] = movies["tags"].apply(lemmatize)

In [304]:
movies.iloc[3].tags

"Following the death of District Attorney Harvey Dent , Batman assumes responsibility for Dent 's crime to protect the late attorney 's reputation and is subsequently hunted by the Gotham City Police Department . Eight year later , Batman encounter the mysterious Selina Kyle and the villainous Bane , a new terrorist leader who overwhelms Gotham 's finest . The Dark Knight resurfaces to protect a city that ha branded him an enemy . Action Crime Drama Thriller ChristianBale MichaelCaine GaryOldman ChristopherNolan"

In [305]:
movies.shape

(4806, 3)

In [ ]:
movies.head()

,movie_id,title,cast,crew,genres,tags
0,19995,Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction]","In the 22nd century , a paraplegic Marine is d..."
1,285,Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Adventure, Fantasy, Action]","Captain Barbossa , long believed to be dead , ..."
2,206647,Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[Action, Adventure, Crime]",A cryptic message from Bond ’ s past sends him...
3,49026,The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Action, Crime, Drama, Thriller]",Following the death of District Attorney Harve...
4,49529,John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[Action, Adventure, ScienceFiction]","John Carter is a war-weary , former military c..."


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(vector)

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [272]:
def get_recommendations_by_title(title, top_n=5):
    try:
        idx = movies[movies['title'].str.lower() == title.lower()].index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        movie_indices = [i[0] for i in sim_scores[1:top_n+1]]
        return movies['title'].iloc[movie_indices].tolist()
    except:
        return ["Movie not found. Try another title."]

In [ ]:
get_recommendations_by_title("The Dark Knight Rises")

['The Dark Knight', 'Batman Forever', 'Batman', 'Batman Begins', 'Batman']

In [ ]:
def recommend_by_genre(genre, top_n=5):
    filtered = movies[movies['tags'].str.contains(genre.lower(), case=False)]
    return filtered['title'].head(top_n).tolist()

In [ ]:
recommend_by_genre("crime")

['Spectre',
 'The Dark Knight Rises',
 'Quantum of Solace',
 'Rush Hour 3',
 'The Dark Knight']

In [ ]:
movies['genres']

,genres
0,"[Action, Adventure, Fantasy, ScienceFiction]"
1,"[Adventure, Fantasy, Action]"
2,"[Action, Adventure, Crime]"
3,"[Action, Crime, Drama, Thriller]"
4,"[Action, Adventure, ScienceFiction]"
...,...
4804,"[Action, Crime, Thriller]"
4805,"[Comedy, Romance]"
4806,"[Comedy, Drama, Romance, TVMovie]"
4807,[]


In [ ]:
movies.isnull().sum()

,0
movie_id,0
title,0
cast,0
crew,0
genres,0
tags,0


In [ ]:
def recommend_by_director(director, top_n=5):
    filtered = movies[movies['tags'].str.contains(director.lower(), case=False)]
    return filtered['title'].head(top_n).tolist()

In [ ]:
recommend_by_director("JamesCameron")

['Avatar', 'Titanic', 'Terminator 2: Judgment Day', 'True Lies', 'The Abyss']

In [ ]:
def recommend_by_actor(actor, top_n=5):
  filtered_Actor = movies[movies['tags'].str.contains(actor.lower(), case = False)]
  return filtered_Actor['title'].head(top_n).tolist()

In [ ]:
recommend_by_actor("christianbale")

['The Dark Knight Rises',
 'Terminator Salvation',
 'The Dark Knight',
 'Batman Begins',
 'Exodus: Gods and Kings']

In [ ]:
def recommend_by_mood(mood, top_n=5):
    sentiment = sia.polarity_scores(mood)['compound']
    if sentiment > 0.1:
        filtered = movies[movies['tags'].str.contains('adventure|comedy|romance|action', case=False)]
    else:
        filtered = movies[movies['tags'].str.contains('thriller|horror|drama|mystery', case=False)]
    return filtered['title'].head(top_n).tolist()


In [ ]:
recommend_by_mood("relax")

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter']

In [ ]:
import pickle
with open("movie.pkl", 'wb') as file:
  pickle.dump(movies, file)

In [ ]:
import pickle
with open("similarity.pkl")